In [0]:
dbutils.library.restartPython() 

In [0]:
import numpy as np
import pandas as pd
#!pip install --upgrade typing-extensions
#!pip install sqlalchemy
from sqlalchemy import create_engine
!pip install dotenv
import dotenv 
import os
#import sqlite3
!pip install psycopg-binary
import psycopg2

In [0]:
#!pip install openpyxl
# pull census data from weldon cooper center


population_2012_by_race = pd.read_excel('https://virginia.box.com/shared/static/i8i5onrkveks849pkky0gwgxlax8d8fe.xlsx')

population_2012_by_hispanic_origin = pd.read_excel('https://virginia.box.com/shared/static/fegrn0p0igzl95snji3ku6edwu0hy3dj.xlsx', skiprows=[0,1,2,3,5,6,7,8,9])[['FIPS','Unnamed: 6']].rename(columns={'Unnamed: 6':'hispanic_population_2022'})

population_2012_by_hispanic_origin

In [0]:
race_pop = pd.read_excel('https://virginia.box.com/shared/static/i8i5onrkveks849pkky0gwgxlax8d8fe.xlsx', skiprows=[0,1,2,3,5,6,7])
race_pop.head(10)


In [0]:
race_pop.drop(columns=[col for col in race_pop.columns if "Unnamed" in col], axis=1,inplace=True)
race_pop.head(10)   

In [0]:
    population_2012_by_hispanic_origin.dtypes

In [0]:
joined_table=pd.merge(race_pop, population_2012_by_hispanic_origin, on='FIPS', validate='one_to_one',indicator='matched')
joined_table.head(10)

In [0]:
joined_table.matched.value_counts()

In [0]:
cases=pd.read_csv('/Workspace/Users/eashan@gmail.com/data1k.csv')
convictions = cases.query('convictions == True')
convictions

In [0]:
joined_table=joined_table.rename({
    'FIPS':'fips',
    'Jurisdiction':'jurisdiction',
    'Total Population':'total_pop',
    'American Indian':'amer_ind_pop',
    'Asian ':'asian_pop',
    'Black':'black_pop',
    'Pacific Islander':'pacific_pop',
    'Two or more races':'twoormore_pop',
    'White':'white_pop',
    'Hispanic':'hisp_pop'},axis=1
)
joined_table

In [0]:
convictions_race = convictions.groupby(['CodeSection','Race','fips']).size().reset_index().rename({0:'count'}, axis=1)

convictions_race

In [0]:
convictions_race_wide =  pd.pivot_table(convictions_race,index=['CodeSection','fips'],values = 'count',columns='Race',fill_value=0).reset_index(); convictions_race_wide

In [0]:
convict_census = pd.merge(convictions_race_wide,joined_table,on='fips',how='outer',validate='many_to_one', indicator = 'matched3')
convict_census

In [0]:
convict_census['matched3'].value_counts()

In [0]:
convict_census[convict_census['matched3'] == 'right_only']

In [0]:
convict_census[convict_census['matched3'] == 'left_only']['fips'].drop_duplicates()

In [0]:
convict_census['total_convictions'] = convict_census['Black'] +convict_census['White'] 

In [0]:
convict_census.query('total_convictions > 1')

In [0]:
convict_census['black_overrep_index']=convict_census['Black']/convict_census['total_convictions']/convict_census['black_pop']/convict_census['total_pop']

In [0]:
convict_census

In [0]:
convict_census = convict_census.dropna(subset=['black_overrep_index']).sort_values(by='black_overrep_index',ascending=False).head(15)
convict_census